In [ ]:
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from model import beta_model, integ_beta_model, inv_beta_model
from utilities import parallel_process
from astropy.table import Table, Column
import numpy as np

In [ ]:
def correct_fluxes(name, outpath):

    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return

    # get the MCMC fits:
    if os.path.isfile(f'{outpath}/{name}/{name}_mcmcfits.txt'):
        mcmcfit = Table.read(f'{outpath}/{name}/{name}_mcmcfits.txt',
                             format='ascii',
                             header_start=0)
        
    # get the spectral fluxes
    filename = f'{outpath}/{name}/spec_files/{name}_xspec.results'

    if os.path.isfile(filename):
        try:
            spec_info = Table.read(filename, format='ascii', header_start=2)
        except ValueError:
            spec_info = Table.read(filename, format='ascii', header_start=0)
            
    # add new columns
    cols = ['FLUX_CORR', 'LUM_CORR', 'CORR']
    for col in cols:
        try:
            spec_info.add_column(Column(data=np.zeros(len(spec_info)), name=col, dtype='<f8'))
        except ValueError:
            pass
    try:
        spec_info.add_column(Column(data=[name.replace('_', ' ')] * len(spec_info), name='FIELD'))
    except ValueError:
        pass
    spec_info['FIELD'] = name.replace('_', ' ')

    if len(spec_info) < 1:
        spec_info.write(filename, format='ascii.commented_header', overwrite=True)
        return
    
    
    # now we have to loop through the detections
    for i, (j, flux, lum) in enumerate(spec_info[['INDEX', 'FLUX', 'LUM']]):

        # Break out the values we are gonna need about the beta model
        So = mcmcfit['So_50'][mcmcfit['ID'] == j].data[0]
        rc = mcmcfit['rc_50'][mcmcfit['ID'] == j].data[0] # arcmin
        beta = mcmcfit['beta_50'][mcmcfit['ID'] == j].data[0]
        bkg = mcmcfit['bg_50'][mcmcfit['ID'] == j].data[0] # cnts/s/arcmin2
        
        # get all the different radii that we will need
        # background
        radius_bkg = inv_beta_model(So, rc, beta, bkg)
        
        # integrate the beta model 0 -> bkg
        zero_bkg = integ_beta_model(radius_bkg, rc=rc, beta=beta)  # cnts
            
        # integrate the beta model bkg -> 5*rc
        bkg_5rc = integ_beta_model(radius_bkg, rc=5*rc, beta=beta)  # cnts
        
        spec_info[i]['CORR'] = bkg_5rc / zero_bkg
        spec_info[i]['FLUX_CORR'] = flux * (1 + bkg_5rc / zero_bkg)
        spec_info[i]['LUM_CORR'] = lum * (1 + bkg_5rc / zero_bkg)
        
        spec_info.write(filename, format='ascii.commented_header', overwrite=True)
        
#         print(So, rc, beta, bkg)
#         print(radius_bkg, rc)
#         print(zero_bkg, bkg_5rc)
        
    return bkg_5rc / zero_bkg
        
        

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')

outpath = './data'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath} for n in data['NAME']]
results = parallel_process(arr, correct_fluxes, use_kwargs=True, n_jobs=1)


In [ ]:
outpath ='./data_full_new'
name = 'PSZ2_G075.08+19.83'
name = 'PSZ2_G283.91+73.87'
# name = 'PSZ1_G121.35-42.47'
# name = 'PSZ2_G057.80+88.00'

correct_fluxes(name, outpath)

In [ ]:
results = [x for x in results if isinstance(x, float)]

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
results = np.array(results)

In [ ]:
plt.hist(results, bins=25)

In [ ]:
np.nanmedian(results)

In [ ]:
np.nanmean(results)

In [ ]:
len(results)

In [ ]:
results